In [1]:
rm -r highway_env.egg-info

In [2]:
rm -r build

In [3]:
rm -r dist

In [4]:
rm -r /Users/analparakh/anaconda3/lib/python3.6/site-packages/highway_env-1.0.dev0-py3.6.egg

rm: /Users/analparakh/anaconda3/lib/python3.6/site-packages/highway_env-1.0.dev0-py3.6.egg: No such file or directory


In [5]:
rm -r highway_env/__pycache__

In [6]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating highway_env.egg-info
writing highway_env.egg-info/PKG-INFO
writing dependency_links to highway_env.egg-info/dependency_links.txt
writing entry points to highway_env.egg-info/entry_points.txt
writing requirements to highway_env.egg-info/requires.txt
writing top-level names to highway_env.egg-info/top_level.txt
writing manifest file 'highway_env.egg-info/SOURCES.txt'
reading manifest file 'highway_env.egg-info/SOURCES.txt'
writing manifest file 'highway_env.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.7-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/highway_env
copying highway_env/interval.py -> build/lib/highway_env
copying highway_env/__init__.py -> build/lib/highway_env
copying highway_env/types.py -> build/lib/highway_env
copying highway_env/utils.py -> build/lib/highway_env
creating build/lib/tests
creating build/lib/tests/envs
copying tests/env

byte-compiling build/bdist.macosx-10.7-x86_64/egg/tests/envs/test_gym.py to test_gym.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/tests/envs/__init__.py to __init__.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/highway_env/vehicle/graphics.py to graphics.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/highway_env/vehicle/kinematics.py to kinematics.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/highway_env/vehicle/controller.py to controller.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/highway_env/vehicle/behavior.py to behavior.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/highway_env/vehicle/__init__.py to __init__.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/highway_env/vehicle/derby.py to derby.cpython-36.pyc
byte-compiling build/bdist.macosx-10.7-x86_64/egg/highway_env/vehicle/uncertainty/estimation.py to estimation.cpython-36.pyc
byte-compiling buil

In [7]:
import gym
import highway_env
import numpy as np

In [8]:
env = gym.make("demolition_derby-multi-agent-v0")
import numpy as np
import random
import math

from collections import deque
import itertools

class ReplayBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        state = np.expand_dims(state, 0)
        next_state = np.expand_dims(next_state, 0)
            
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action, reward, np.concatenate(next_state), done

    def __len__(self):
        return len(self.buffer)

class ReservoirBuffer(object):
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action):
        state = np.expand_dims(state, 0)
        self.buffer.append((state, action))
    
    def sample(self, batch_size):
        # Efficient Reservoir Sampling
        # http://erikerlandson.github.io/blog/2015/11/20/very-fast-reservoir-sampling/
        n = len(self.buffer)
        reservoir = list(itertools.islice(self.buffer, 0, batch_size))
        threshold = batch_size * 4
        idx = batch_size
        while (idx < n and idx <= threshold):
            m = random.randint(0, idx)
            if m < batch_size:
                reservoir[m] = self.buffer[idx]
            idx += 1
        
        while (idx < n):
            p = float(batch_size) / idx
            u = random.random()
            g = math.floor(math.log(u) / math.log(1 - p))
            idx = idx + g
            if idx < n:
                k = random.randint(0, batch_size - 1)
                reservoir[k] = self.buffer[idx]
            idx += 1
        state, action = zip(*random.sample(self.buffer, batch_size))
        return np.concatenate(state), action
    
    def __len__(self):
        return len(self.buffer)

In [9]:
import math
import os
import datetime
import time
import pathlib
import random

import torch
import numpy as np

def update_target(current_model, target_model):
    target_model.load_state_dict(current_model.state_dict())

def epsilon_scheduler(eps_start, eps_final, eps_decay):
    def function(frame_idx):
        return eps_final + (eps_start - eps_final) * math.exp(-1. * frame_idx / eps_decay)
    return function

def create_log_dir(args):
    log_dir = ""
    log_dir = log_dir + "{}-".format(args.env)
#     if args.negative:
#         log_dir = log_dir + "negative-"
    if args.multi_step != 1:
        log_dir = log_dir + "{}-step-".format(args.multi_step)
#     if args.dueling:
#         log_dir = log_dir + "dueling-"
    log_dir = log_dir + "dqn-{}".format(args.save_model)

    now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
    log_dir = log_dir + now
    log_dir = os.path.join("runs", log_dir)
    return log_dir

def print_log(frame, prev_frame, prev_time, rewards, length_list, rl_losses, sl_losses):
    fps = (frame - prev_frame) / (time.time() - prev_time)
    p1_avg_reward, p2_avg_reward = (np.mean(rewards[0]), np.mean(rewards[1]))
    if len(rl_losses[0]) != 0:
        p1_avg_rl_loss, p2_avg_rl_loss = (np.mean(rl_losses[0]), np.mean(rl_losses[1]))
        p1_avg_sl_loss, p2_avg_sl_loss = (np.mean(sl_losses[0]), np.mean(sl_losses[1]))
    else:
        p1_avg_rl_loss, p2_avg_rl_loss = 0., 0.
        p1_avg_sl_loss, p2_avg_sl_loss = 0., 0.

    avg_length = np.mean(length_list)

    print("Frame: {:<8} FPS: {:.2f} Avg. Tagging Interval Length: {:.2f}".format(frame, fps, avg_length))
    print("Player 1 Avg. Reward: {:.2f} Avg. RL/SL Loss: {:.2f}/{:.2f}".format(
        p1_avg_reward, p1_avg_rl_loss, p1_avg_sl_loss))
    print("Player 2 Avg. Reward: {:.2f} Avg. RL/SL Loss: {:.2f}/{:.2f}".format(
        p2_avg_reward, p2_avg_rl_loss, p2_avg_sl_loss))

def print_args(args):
    print(' ' * 26 + 'Options')
    for k, v in vars(args).items():
        print(' ' * 26 + k + ': ' + str(v))

def save_model(models, policies, args):
    fname = ""
    fname += "{}-".format(args.env)
#     if args.negative:
#         fname += "negative-"
    if args.multi_step != 1:
        fname += "{}-step-".format(args.multi_step)
#     if args.dueling:
#         fname += "dueling-"
    fname += "dqn-{}.pth".format(args.save_model)
    fname = os.path.join("models", fname)

    pathlib.Path('models').mkdir(exist_ok=True)
    torch.save({
        'p1_model': models['p1'].state_dict(),
        'p2_model': models['p2'].state_dict(),
        'p1_policy': policies['p1'].state_dict(),
        'p2_policy': policies['p2'].state_dict(),
    }, fname)

def load_model(models, policies, load_model,env,negative):
    if args.load_model is not None:
        fname = os.path.join("models", load_model)
        fname += ".pth"
    else:
        fname = ""
        fname += "{}-".format(env)
#         if negative:
#             fname += "negative-"
        if multi_step != 1:
            fname += "{}-step-".format(args.multi_step)
#         if args.dueling:
#             fname += "dueling-"
        fname += "dqn-{}.pth".format(args.save_model)
        fname = os.path.join("models", fname)

    # Hack to load models saved with GPU
    if args.device == torch.device("cpu"):
        map_location = lambda storage, loc: storage
    else:
        map_location = None
    
    if not os.path.exists(fname):
        raise ValueError("No model saved with name {}".format(fname))

    checkpoint = torch.load(fname, map_location)
    models['p1'].load_state_dict(checkpoint['p1_model'])
    models['p2'].load_state_dict(checkpoint['p2_model'])
    policies['p1'].load_state_dict(checkpoint['p1_policy'])
    policies['p2'].load_state_dict(checkpoint['p2_policy'])


def set_global_seeds(seed):
    try:
        import torch
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
    except ImportError:
        pass

    np.random.seed(seed)
    random.seed(seed)


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import random
import math

from functools import partial

def DQN(input_dim, output_dim, hidden_dim):
  
    model = DQNBase(input_dim, output_dim, hidden_dim)
    return model

class DQNBase(nn.Module):
    """
    Basic DQN
    parameters
    ---------
    env         environment(openai gym)
    """
    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int) -> None:
        super(DQNBase, self).__init__()
        self.input_shape=input_dim
        self.num_actions=output_dim
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 32), torch.nn.ReLU()
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.Linear(32,32), torch.nn.ReLU()
        )
        # self.layer3 = torch.nn.Sequential(
        #     torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.ReLU()
        # )
        self.final = torch.nn.Linear(32, output_dim)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Returns a Q_value
        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, input_dim)
        Returns:
            torch.Tensor: Q_value, 2-D tensor of shape (n, output_dim)
        """
        x = x.float()
        x = self.layer1(x)
        x = self.layer2(x)
        #x = self.layer3(x)
        x = self.final(x)
        return x
    

    
    def act(self, state, epsilon)-> np.array:
        """
        Parameters
        ----------
        state       torch.Tensor with appropritate device type
        epsilon     epsilon for epsilon-greedy
        """

        
        states = torch.tensor(state)
        if np.random.rand() > epsilon:
            action = self.forward(states).detach().numpy()
        else:
            action = np.random.rand(2)*2.0 - 1.0
       
        return action



class Policy(DQNBase):
    """
    Policy with only actors. This is used in supervised learning for NFSP.
    """
    def __init__(self,state_num,action_num,hidden_num):
        super(Policy, self).__init__(state_num,action_num,hidden_num)

    def act(self, states: np.ndarray) -> np.array:
        states = torch.tensor(states)
    
        action = self.forward(states).detach().numpy()
        return action


# class cReLU(nn.Module):
#     def forward(self, x):
#         return torch.cat((F.relu(x), F.relu(-x)), dim=1)

# class Flatten(nn.Module):
#     def forward(self, x):
#         return x.view(x.size(0), -1)

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import gym
import torch.optim as optim
import random
import os
import time

from matplotlib import pyplot

import numpy as np
import gym
import highway_env


import torch.optim as optim

from collections import deque



# derby_radius = 25.
# max_speed = 10.
# env = gym.make("demolition_derby-multi-agent-v0")
# env.config["derby_radius"]=derby_radius
# env.config["observation"]["observation_config"]["features_range"]["x"]=[-derby_radius,derby_radius]
# env.config["observation"]["observation_config"]["features_range"]["y"]=[-derby_radius,derby_radius]
# env.config["observation"]["observation_config"]["features_range"]["vx"]=[-max_speed,max_speed]
# env.config["observation"]["observation_config"]["features_range"]["vy"]=[-max_speed,max_speed]
# env.config["simulation_frequency"] = 30
# env.config["duration"] = 120.0
# env.reset()
# env.road.vehicles[0].MAX_SPEED=max_speed
# env.road.vehicles[1].MAX_SPEED=max_speed
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# try some random action


def train(env, state_num,action_num,hidden_layer,device,eps_start, eps_final, eps_decay,buffer_size,multi_step,lr,max_frames,eta,gamma,max_tag_interval,rl_start,sl_start,batch_size,train_freq,negative,_update_target,evaluation_interval,render):
    # RL Model for Player 1
    p1_current_model = DQN(state_num,action_num,16).to(device)
    p1_target_model = DQN(state_num,action_num,16).to(device)
    update_target(p1_current_model, p1_target_model)

    # RL Model for Player 2
    p2_current_model = DQN(state_num,action_num,16).to(device)
    p2_target_model = DQN(state_num,action_num,16).to(device)
    update_target(p2_current_model, p2_target_model)

    # SL Model for Player 1, 2
    p1_policy = Policy(state_num,action_num,16).to(device)
    p2_policy = Policy(state_num,action_num,16).to(device)

    # if args.load_model and os.path.isfile(args.load_model):
    #     load_model(models={"p1": p1_current_model, "p2": p2_current_model},
    #                policies={"p1": p1_policy, "p2": p2_policy}, args=args)

    epsilon_by_frame = epsilon_scheduler(eps_start, eps_final, eps_decay)

    # Replay Buffer for Reinforcement Learning - Best Response
    p1_replay_buffer = ReplayBuffer(buffer_size)
    p2_replay_buffer = ReplayBuffer(buffer_size)

    # Reservoir Buffer for Supervised Learning - Average Strategy
    # TODO(Aiden): How to set buffer size of SL?
    p1_reservoir_buffer = ReservoirBuffer(buffer_size)
    p2_reservoir_buffer = ReservoirBuffer(buffer_size)

    # Deque data structure for multi-step learning
    p1_state_deque = deque(maxlen=multi_step)
    p1_reward_deque = deque(maxlen=multi_step)
    p1_action_deque = deque(maxlen=multi_step)

    p2_state_deque = deque(maxlen=multi_step)
    p2_reward_deque = deque(maxlen=multi_step)
    p2_action_deque = deque(maxlen=multi_step)

    # RL Optimizer for Player 1, 2
    p1_rl_optimizer = optim.Adam(p1_current_model.parameters(), lr=lr)
    p2_rl_optimizer = optim.Adam(p2_current_model.parameters(), lr=lr)

    # SL Optimizer for Player 1, 2
    # TODO(Aiden): Is it necessary to seperate learning rate for RL/SL?
    p1_sl_optimizer = optim.Adam(p1_policy.parameters(), lr=lr)
    p2_sl_optimizer = optim.Adam(p2_policy.parameters(), lr==lr)

    # Logging
    length_list = []
    p1_reward_list, p1_rl_loss_list, p1_sl_loss_list = [], [], []
    p2_reward_list, p2_rl_loss_list, p2_sl_loss_list = [], [], []
    p1_episode_reward, p2_episode_reward = 0, 0
    tag_interval_length = 0
    prev_time = time.time()
    prev_frame = 1

    # Main Loop
    next_state = env.reset()
      # next_state = next_state[0][1,:]
    x_new = next_state[0][1,0] * np.cos(next_state[0][0,4]) + next_state[0][1,1] * np.sin(next_state[0][0,4])
    y_new = next_state[0][1,1] * np.cos(next_state[0][0,4]) - next_state[0][1,0] * np.sin(next_state[0][0,4])
    vx_new = next_state[0][1,2] * np.cos(next_state[0][0,4]) + next_state[0][1,3] * np.sin(next_state[0][0,4])
    vy_new = next_state[0][1,3] * np.cos(next_state[0][0,4]) - next_state[0][1,2] * np.sin(next_state[0][0,4])
        
    p1_state= np.array([x_new,y_new,vx_new,vy_new,next_state[0][1,4]])
#next_state[1][0,0],next_state[1][0,1],
    x_new = next_state[1][1,0] * np.cos(next_state[1][0,4]) + next_state[1][1,1] * np.sin(next_state[1][0,4])
    y_new = next_state[1][1,1] * np.cos(next_state[1][0,4]) - next_state[1][1,0] * np.sin(next_state[1][0,4])
    vx_new = next_state[1][1,2] * np.cos(next_state[1][0,4]) + next_state[1][1,3] * np.sin(next_state[1][0,4])
    vy_new = next_state[1][1,3] * np.cos(next_state[1][0,4]) - next_state[1][1,2] * np.sin(next_state[1][0,4])
        
    p2_state = np.array([x_new,y_new,vx_new,vy_new,next_state[1][1,4]])
#next_state[1][0,0],next_state[1][0,1],

    for frame_idx in range(1, max_frames + 1):
        env.render()
        is_best_response = False
        # TODO(Aiden):
        # Action should be decided by a combination of Best Response and Average Strategy
        if random.random() > eta:
            p1_action = p1_policy.act(torch.FloatTensor(p1_state).to(device))
            p2_action = p2_policy.act(torch.FloatTensor(p1_state).to(device))
        else:
            is_best_response = True
            epsilon = epsilon_by_frame(frame_idx)
            p1_action = p1_current_model.act(torch.FloatTensor(p1_state).to(device), epsilon)
            p2_action = p2_current_model.act(torch.FloatTensor(p2_state).to(device), epsilon)
        
        actions = (p1_action, p2_action)
  
        next_next_state, _, done, info = env.step(actions)
    
        reward=info["agents_rewards"]
        x_new = next_next_state[0][1,0] * np.cos(next_next_state[0][0,4]) + next_next_state[0][1,1] * np.sin(next_next_state[0][0,4])
        y_new = next_next_state[0][1,1] * np.cos(next_next_state[0][0,4]) - next_next_state[0][1,0] * np.sin(next_next_state[0][0,4])
        vx_new = next_next_state[0][1,2] * np.cos(next_next_state[0][0,4]) + next_next_state[0][1,3] * np.sin(next_next_state[0][0,4])
        vy_new = next_next_state[0][1,3] * np.cos(next_next_state[0][0,4]) - next_next_state[0][1,2] * np.sin(next_next_state[0][0,4])
            
        p1_next_state= np.array([x_new,y_new,vx_new,vy_new,next_next_state[0][1,4]])
#next_next_state[0][0,0],next_next_state[0][0,1],
        x_new = next_next_state[1][1,0] * np.cos(next_next_state[1][0,4]) + next_next_state[1][1,1] * np.sin(next_next_state[1][0,4])
        y_new = next_next_state[1][1,1] * np.cos(next_next_state[1][0,4]) - next_next_state[1][1,0] * np.sin(next_next_state[1][0,4])
        vx_new = next_next_state[1][1,2] * np.cos(next_next_state[1][0,4]) + next_next_state[1][1,3] * np.sin(next_next_state[1][0,4])
        vy_new = next_next_state[1][1,3] * np.cos(next_next_state[1][0,4]) - next_next_state[1][1,2] * np.sin(next_next_state[1][0,4])
            
        p2_next_state = np.array([x_new,y_new,vx_new,vy_new,next_next_state[1][1,4]])
        print("next_state" , next_next_state)
        #print("next_state velocity ",next_next_state[0][0,2],next_next_state[0][0,3],next_next_state[1][0,2],next_next_state[1][0,3])
#next_next_state[1][0,0],next_next_state[1][0,1],
        # Save current state, reward, action to deque for multi-step learning
        p1_state_deque.append(p1_state)
        p2_state_deque.append(p2_state)
        
        p1_reward = reward[0] - 1 if negative else reward[0]
        p2_reward = reward[1] - 1 if negative else reward[1]
        p1_reward_deque.append(p1_reward)
        p2_reward_deque.append(p2_reward)

        p1_action_deque.append(p1_action)
        p2_action_deque.append(p2_action)

        # Store (state, action, reward, next_state) to Replay Buffer for Reinforcement Learning
        if len(p1_state_deque) == multi_step or done:
            n_reward = multi_step_reward(p1_reward_deque, gamma)
            n_state = p1_state_deque[0]
            n_action = p1_action_deque[0]
            p1_replay_buffer.push(n_state, n_action, n_reward, p1_next_state, np.float32(done))

            n_reward = multi_step_reward(p2_reward_deque, gamma)
            n_state = p2_state_deque[0]
            n_action = p2_action_deque[0]
            p2_replay_buffer.push(n_state, n_action, n_reward, p2_next_state, np.float32(done))
        
        # Store (state, action) to Reservoir Buffer for Supervised Learning
        if is_best_response:
            p1_reservoir_buffer.push(p1_state, p1_action)
            p2_reservoir_buffer.push(p2_state, p2_action)

        (p1_state, p2_state) = (p1_next_state, p2_next_state)

        # Logging
        p1_episode_reward += p1_reward
        p2_episode_reward += p2_reward
        tag_interval_length += 1

        if info is not None:
            length_list.append(tag_interval_length)
            tag_interval_length = 0

        # Episode done. Reset environment and clear logging records
        if done or tag_interval_length >= max_tag_interval:
            
            next_state = env.reset()
              # next_state = next_state[0][1,:]
            x_new = next_state[0][1,0] * np.cos(next_state[0][0,4]) + next_state[0][1,1] * np.sin(next_state[0][0,4])
            y_new = next_state[0][1,1] * np.cos(next_state[0][0,4]) - next_state[0][1,0] * np.sin(next_state[0][0,4])
            vx_new = next_state[0][1,2] * np.cos(next_state[0][0,4]) + next_state[0][1,3] * np.sin(next_state[0][0,4])
            vy_new = next_state[0][1,3] * np.cos(next_state[0][0,4]) - next_state[0][1,2] * np.sin(next_state[0][0,4])
                
            p1_state= np.array([x_new,y_new,vx_new,vy_new,next_state[0][1,4]])
#next_state[0][0,0],next_state[0][0,1],
            x_new = next_state[1][1,0] * np.cos(next_state[1][0,4]) + next_state[1][1,1] * np.sin(next_state[1][0,4])
            y_new = next_state[1][1,1] * np.cos(next_state[1][0,4]) - next_state[1][1,0] * np.sin(next_state[1][0,4])
            vx_new = next_state[1][1,2] * np.cos(next_state[1][0,4]) + next_state[1][1,3] * np.sin(next_state[1][0,4])
            vy_new = next_state[1][1,3] * np.cos(next_state[1][0,4]) - next_state[1][1,2] * np.sin(next_state[1][0,4])
                
            p2_state = np.array([x_new,y_new,vx_new,vy_new,next_state[1][1,4]])

#next_state[1][0,0],next_state[1][0,1],
      
            p1_reward_list.append(p1_episode_reward)
            p2_reward_list.append(p2_episode_reward)
            # writer.add_scalar("p1/episode_reward", p1_episode_reward, frame_idx)
            # writer.add_scalar("p2/episode_reward", p2_episode_reward, frame_idx)
            # writer.add_scalar("data/tag_interval_length", tag_interval_length, frame_idx)
            p1_episode_reward, p2_episode_reward, tag_interval_length = 0, 0, 0
            p1_state_deque.clear(), p2_state_deque.clear()
            p1_reward_deque.clear(), p2_reward_deque.clear()
            p1_action_deque.clear(), p2_action_deque.clear()

        if (len(p1_replay_buffer) > rl_start and
            len(p1_reservoir_buffer) > sl_start and
            frame_idx % train_freq == 0):

            # Update Best Response with Reinforcement Learning
            loss = compute_rl_loss(p1_current_model, p1_target_model, p1_replay_buffer, p1_rl_optimizer,batch_size,device,multi_step,gamma)
            p1_rl_loss_list.append(loss.item())
            

            loss = compute_rl_loss(p2_current_model, p2_target_model, p2_replay_buffer, p2_rl_optimizer,batch_size,device,multi_step,gamma)
            p2_rl_loss_list.append(loss.item())
          

            # Update Average Strategy with Supervised Learning
            loss = compute_sl_loss(p1_policy, p1_reservoir_buffer, p1_sl_optimizer, batch_size,device)
            p1_sl_loss_list.append(loss.item())
         

            loss = compute_sl_loss(p2_policy, p2_reservoir_buffer, p2_sl_optimizer,batch_size,device)
            p2_sl_loss_list.append(loss.item())
         
        

        if frame_idx % _update_target == 0:
            update_target(p1_current_model, p1_target_model)
            update_target(p2_current_model, p2_target_model)


        # Logging and Saving models
        if frame_idx % evaluation_interval == 0:
            print_log(frame_idx, prev_frame, prev_time, (p1_reward_list, p2_reward_list), length_list,
                      (p1_rl_loss_list, p2_rl_loss_list), (p1_sl_loss_list, p2_sl_loss_list))
            p1_reward_list.clear(), p2_reward_list.clear(), length_list.clear()
            p1_rl_loss_list.clear(), p2_rl_loss_list.clear()
            p1_sl_loss_list.clear(), p2_sl_loss_list.clear()
            prev_frame = frame_idx
            prev_time = time.time()
            # save_model(models={"p1": p1_current_model, "p2": p2_current_model},
            #            policies={"p1": p1_policy, "p2": p2_policy}, args=args)
        
        # Render if rendering argument is on
     

        # save_model(models={"p1": p1_current_model, "p2": p2_current_model},
        #             policies={"p1": p1_policy, "p2": p2_policy}, args=args)

def compute_sl_loss(policy, reservoir_buffer, optimizer,batch_size,device):
    state, action = reservoir_buffer.sample(batch_size)

    state = torch.FloatTensor(np.float32(state)).to(device)
    action = torch.LongTensor(action).to(device)

    probs = policy(state)
    probs_with_actions = probs.gather(1, action.unsqueeze(1))
    log_probs = probs_with_actions.log()

    loss = -1 * log_probs.mean()
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss


def compute_rl_loss(current_model, target_model, replay_buffer, optimizer, batch_size,device,multi_step,gamma):
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)
    weights = torch.ones(batch_size)

    state = torch.FloatTensor(np.float32(state)).to(device)
    next_state = torch.FloatTensor(np.float32(next_state)).to(device)
    action = torch.LongTensor(action).to(device)
    reward = torch.FloatTensor(reward).to(device)
    done = torch.FloatTensor(done).to(device)
    weights = torch.FloatTensor(weights).to(device)

    # Q-Learning with target network
    q_values = current_model(state)
    target_next_q_values = target_model(next_state)

    q_value = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
    next_q_value = target_next_q_values.max(1)[0]
    expected_q_value = reward + (gamma ** multi_step) * next_q_value * (1 - done)

    # Huber Loss
    loss = F.smooth_l1_loss(q_value, expected_q_value.detach(), reduction='none')
    loss = (loss * weights).mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss

def multi_step_reward(rewards, gamma):
    ret = 0.
    for idx, reward in enumerate(rewards):
        ret += reward * (gamma ** idx)
    return ret

In [12]:
state_num=5
action_num=2
hidden_layer=16
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
derby_radius = 15
max_speed = 10.
env = gym.make("demolition_derby-multi-agent-v0")
env.config["derby_radius"]=derby_radius
env.config["observation"]["observation_config"]["features_range"]["x"]=[-derby_radius,derby_radius]
env.config["observation"]["observation_config"]["features_range"]["y"]=[-derby_radius,derby_radius]
env.config["observation"]["observation_config"]["features_range"]["vx"]=[-max_speed,max_speed]
env.config["observation"]["observation_config"]["features_range"]["vy"]=[-max_speed,max_speed]
env.config["simulation_frequency"] = 30
env.config["duration"] = 120.0
env.reset()
env.road.vehicles[0].MAX_SPEED=max_speed
env.road.vehicles[1].MAX_SPEED=max_speed
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
eps_start=1
eps_final=.01
eps_decay=30000
buffer_size=100000
multi_step=1
lr=1e-4
max_frames=1400000
eta=.1
gamma=.99
max_tag_interval=1000
rl_start=10000
sl_start=1000
batch_size=32
train_freq=1
negative=False
_update_target=1000
evaluation_interval=10000
render=True
train(env, state_num,action_num,hidden_layer,device,eps_start, eps_final, eps_decay,buffer_size,multi_step,lr,max_frames,eta,gamma,max_tag_interval,rl_start,sl_start,batch_size,train_freq,negative,_update_target,evaluation_interval,render)

/Users/analparakh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


new rewards
next_state (array([[ 0.5219899 , -0.49828954,  0.01102432, -0.07428865,  1.7181196 ],
       [-0.77716118,  0.40070105, -0.10334756,  0.08799321,  6.13582033]]), array([[-0.25517128, -0.09758849, -0.09232324,  0.01370456,  6.13582033],
       [ 0.77716118, -0.40070105,  0.10334756, -0.08799321,  1.7181196 ]]))
new rewards
next_state (array([[ 0.5379065 , -0.57035674,  0.01602748, -0.14791369,  1.67873217],
       [-0.88416597,  0.49006786, -0.20272833,  0.1698049 ,  6.16646539]]), array([[-0.34625947, -0.08028889, -0.18670085,  0.02189121,  6.16646539],
       [ 0.88416597, -0.49006786,  0.20272833, -0.1698049 ,  1.67873217]]))
new rewards
next_state (array([[ 0.55804953, -0.69214102,  0.0097426 , -0.22389942,  1.61428216],
       [-1.05897632,  0.63455543, -0.29335641,  0.24292117,  6.21621642]]), array([[-0.50092679, -0.05758559, -0.28361381,  0.01902175,  6.21621642],
       [ 1.05897632, -0.63455543,  0.29335641, -0.24292117,  1.61428216]]))
new rewards
next_state (arra

/Users/analparakh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


new rewards
next_state (array([[ 0.50541098, -0.70513313,  0.01393984, -0.17603464,  1.64981948],
       [-1.58106797,  0.66450518, -0.63267388, -0.14351153,  6.75990739]]), array([[-1.075657  , -0.04062795, -0.61873404, -0.31954617,  6.75990739],
       [ 1.58106797, -0.66450518,  0.63267388,  0.14351153,  1.64981948]]))
new rewards
next_state (array([[ 4.85674377e-01, -6.98778499e-01,  5.76050364e-03,
        -9.76083814e-02,  1.62974444e+00],
       [-1.34158202e+00,  6.53615090e-01, -1.13152651e-01,
         4.04269022e-02,  6.77245851e+00]]), array([[-0.85590765, -0.04516341, -0.10739215, -0.05718148,  6.77245851],
       [ 1.34158202, -0.65361509,  0.11315265, -0.0404269 ,  1.62974444]]))
new rewards
next_state (array([[ 4.75018625e-01, -7.05996068e-01,  3.79498648e-03,
        -9.73165803e-02,  1.60977288e+00],
       [-1.32651378e+00,  6.48104192e-01, -9.97182963e-02,
         4.47959495e-02,  6.78412811e+00]]), array([[-0.85149515, -0.05789188, -0.09592331, -0.05252063,  6.784

new rewards
next_state (array([[ 0.06266895, -0.84728742, -0.04739959, -0.06500114,  0.94073297],
       [-0.42683808,  0.80059964, -0.31538424, -0.07604279,  6.65398398]]), array([[-0.36416913, -0.04668778, -0.36278383, -0.14104392,  6.65398398],
       [ 0.42683808, -0.80059964,  0.31538424,  0.07604279,  0.94073297]]))
new rewards
next_state (array([[ 0.04894224, -0.84869947, -0.04944843, -0.0654124 ,  0.92349882],
       [-0.68727196,  0.69839453, -0.38838713, -0.16327486,  6.76452365]]), array([[-0.63832973, -0.15030494, -0.43783556, -0.22868726,  6.76452365],
       [ 0.68727196, -0.69839453,  0.38838713,  0.16327486,  0.92349882]]))
new rewards
next_state (array([[ 0.03531032, -0.84959471, -0.050818  , -0.06468129,  0.90485588],
       [-0.99656676,  0.52919974, -0.44520904, -0.28250711,  6.89386996]]), array([[-0.96125644, -0.32039498, -0.49602704, -0.3471884 ,  6.89386996],
       [ 0.99656676, -0.52919974,  0.44520904,  0.28250711,  0.90485588]]))
new rewards
next_state (arra

...

new rewards
next_state (array([[-0.08084955,  0.5319584 , -0.09882516, -0.07660135,  3.80097763],
       [-0.03476536, -1.20373021, -0.00512637, -0.20668792,  1.21910637]]), array([[-0.11561491, -0.67177181, -0.10395152, -0.28328926,  1.21910637],
       [ 0.03476536,  1.20373021,  0.00512637,  0.20668792,  3.80097763]]))
new rewards
next_state (array([[-0.1357947 ,  0.48433164, -0.07159291, -0.05779028,  3.82071253],
       [-0.05339709, -1.36220166, -0.06176714, -0.27294472,  1.18751436]]), array([[-0.18919178, -0.87787002, -0.13336005, -0.33073501,  1.18751436],
       [ 0.05339709,  1.36220166,  0.06176714,  0.27294472,  3.82071253]]))
new rewards
next_state (array([[-0.173133  ,  0.45062224, -0.04441547, -0.0369324 ,  3.83526045],
       [-0.00945688, -1.27279848,  0.02270203, -0.01646078,  1.18455265]]), array([[-0.18258988, -0.82217625, -0.02171344, -0.05339318,  1.18455265],
       [ 0.00945688,  1.27279848, -0.02270203,  0.01646078,  3.83526045]]))
new rewards
next_state (arra

new rewards
next_state (array([[ 0.13743882,  0.66203882, -0.54185763, -0.13261108,  3.38160914],
       [-0.27300564, -1.35340377,  0.65865796,  0.27072542,  0.86881646]]), array([[-0.13556682, -0.69136496,  0.11680033,  0.13811434,  0.86881646],
       [ 0.27300564,  1.35340377, -0.65865796, -0.27072542,  3.38160914]]))
new rewards
next_state (array([[-0.19461075,  0.53402947, -0.47366941, -0.18615384,  3.51605324],
       [ 0.12336928, -1.14356555,  0.55569081,  0.28658875,  0.88597771]]), array([[-0.07124147, -0.60953608,  0.0820214 ,  0.1004349 ,  0.88597771],
       [-0.12336928,  1.14356555, -0.55569081, -0.28658875,  3.51605324]]))
new rewards
next_state (array([[-0.4820181 ,  0.38218488, -0.41129801, -0.21753074,  3.628083  ],
       [ 0.45282129, -0.93667542,  0.45892297,  0.2772064 ,  0.89723728]]), array([[-0.02919681, -0.55449054,  0.04762497,  0.05967566,  0.89723728],
       [-0.45282129,  0.93667542, -0.45892297, -0.2772064 ,  3.628083  ]]))
new rewards
next_state (arra

new rewards
next_state (array([[-0.82952117, -0.0294318 , -0.17043082, -0.15152697,  3.86834287],
       [ 0.42849106, -0.7046913 ,  0.13244045,  0.11676531,  0.74104776]]), array([[-0.40103011, -0.7341231 , -0.03799037, -0.03476167,  0.74104776],
       [-0.42849106,  0.7046913 , -0.13244045, -0.11676531,  3.86834287]]))
new rewards
next_state (array([[-0.81970958, -0.02203433,  0.02391605,  0.02113988,  3.86545259],
       [ 0.41394452, -0.71000418, -0.06302071, -0.05668309,  0.73772429]]), array([[-0.40576505, -0.73203852, -0.03910466, -0.03554321,  0.73772429],
       [-0.41394452,  0.71000418,  0.06302071,  0.05668309,  3.86545259]]))
new rewards
next_state (array([[-0.70555274,  0.03875296,  0.19763817,  0.3110299 ,  4.14632355],
       [ 0.3465877 , -0.72499061, -0.08081066, -0.20810465,  0.72221914]]), array([[-0.35896504, -0.68623765,  0.11682752,  0.10292525,  0.72221914],
       [-0.3465877 ,  0.72499061,  0.08081066,  0.20810465,  4.14632355]]))
new rewards
next_state (arra

new rewards
next_state (array([[ 0.0289702 ,  0.92170771,  0.24198106,  0.18496466,  3.79423146],
       [ 0.68133343, -0.54902086, -0.28101026, -0.23749686,  0.93181772]]), array([[ 0.71030363,  0.37268686, -0.03902919, -0.0525322 ,  0.93181772],
       [-0.68133343,  0.54902086,  0.28101026,  0.23749686,  3.79423146]]))
new rewards
next_state (array([[ 0.02552934,  0.8442942 ,  0.01845157,  0.01406084,  3.79275523],
       [ 0.64881041, -0.52397327, -0.09606896, -0.11482931,  0.91445769]]), array([[ 0.67433975,  0.32032093, -0.07761739, -0.10076847,  0.91445769],
       [-0.64881041,  0.52397327,  0.09606896,  0.11482931,  3.79275523]]))
new rewards
next_state (array([[ 0.03123838,  0.84426561,  0.02021482,  0.01534729,  3.79096202],
       [ 0.58179856, -0.60824681, -0.13828646, -0.1610375 ,  0.88972823]]), array([[ 0.61303694,  0.2360188 , -0.11807164, -0.14569021,  0.88972823],
       [-0.58179856,  0.60824681,  0.13828646,  0.1610375 ,  3.79096202]]))
new rewards
next_state (arra

RuntimeError: Index tensor must have the same number of dimensions as input tensor

In [ ]:
import argparse
import torch

def get_args():
    parser = argparse.ArgumentParser(description='DQN')

    # Basic Arguments
    parser.add_argument('--seed', type=int, default=1122,
                        help='Random seed')
    parser.add_argument('--batch-size', type=int, default=32,
                        help='Batch size')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')

    # Training Arguments
    parser.add_argument('--max-frames', type=int, default=1400000,
                        help='Number of frames to train')
    parser.add_argument('--buffer-size', type=int, default=100000,
                        help='Maximum memory buffer size')
    parser.add_argument('--update-target', type=int, default=1000,
                        help='Interval of target network update')
    parser.add_argument('--train-freq', type=int, default=1,
                        help='Number of steps between optimization step')
    parser.add_argument('--gamma', type=float, default=0.99,
                        help='Discount factor')
    parser.add_argument('--eta', type=float, default=0.1,
                        help='Anticipatory Parameter for NFSP')
    parser.add_argument('--rl-start', type=int, default=10000,
                        help='How many steps of the model to collect transitions for before RL starts')
    parser.add_argument('--sl-start', type=int, default=1000,
                        help='How many steps of the model to collect transitions for before SL starts')

    # Algorithm Arguments
    parser.add_argument('--dueling', action='store_true',
                        help='Enable Dueling Network')
    parser.add_argument('--multi-step', type=int, default=1,
                        help='N-Step Learning')

    # Environment Arguments
    parser.add_argument('--env', type=str, default='LaserTag-small2-v0',
                        help='Environment Name')
    parser.add_argument('--negative', action='store_true', default=False,
                        help='Give negative(-1) reward for not done.')

    # Evaluation Arguments
    parser.add_argument('--load-model', type=str, default=None,
                        help='Pretrained model name to load (state dict)')
    parser.add_argument('--save-model', type=str, default='model',
                        help='Pretrained model name to save (state dict)')
    parser.add_argument('--evaluate', action='store_true',
                        help='Evaluate only')
    parser.add_argument('--render', action='store_true',
                        help='Render evaluation agent')
    parser.add_argument('--evaluation-interval', type=int, default=10000,
                        help='Frames for evaluation interval')

    # Optimization Arguments
    parser.add_argument('--lr', type=float, default=1e-4,
                        help='Learning rate')
    parser.add_argument('--max-tag-interval', type=int, default=1000,
                        help='Maximum length of interval between tagging twice to prevent from non-action')
    parser.add_argument('--eps-start', type=float, default=1.0,
                        help='Start value of epsilon')
    parser.add_argument('--eps-final', type=float, default=0.01,
                        help='Final value of epsilon')
    parser.add_argument('--eps-decay', type=int, default=30000,
                        help='Adjustment parameter for epsilon')

    args = parser.parse_args()
    args.cuda = not args.no_cuda and torch.cuda.is_available()
    args.device = torch.device("cuda" if args.cuda else "cpu")

    return args

In [ ]:
import numpy as np

In [ ]:
np.random.rand(2)*2.0 - 1.0

In [ ]:
3.37924194-2*np.pi

In [ ]:
Yellow: (array([[-0.83560731, -0.00820829,  0.03886867, -0.00983   ,  2.89388376],
        [ 1.7642926 ,  0.12082875,  0.11393011, -0.0645288 , -3.59449947]]), 
Green : array([[ 0.92868529,  0.11262047,  0.15279877, -0.07435879, -3.59449947],
       [-1.7642926 , -0.12082875, -0.11393011,  0.0645288 ,  2.89388376]]))